# 데이터 분석 알고리즘

- 데이터 분석 알고리즘
  - 머신러닝/딥러닝 분야에서 딥러닝 다음으로 부스팅(boosting) 알고리즘이 핵심적으로 사용됨
  - 선형회귀나 로지스틱 회귀는 가장 대중적인 알고리즘이고, 그 다음이 의사결정트리와 앙상블 계열 알고리즘, 딥러닝


# 앙상블의 개념

- 앙상블(ensemble)
  - 여러 개의 알고리즘들이 하나의 값을 예측하는 기법을 통칭하여 말함
  - 회귀 문제에서는 가중 평균이나 단순 평균을 구하는 방식으로 Y값을 예측
- 메타 분류기(meta-classifier)라고도 부름
  - 메타(meta)는 일종의 상위 또는 추상화라는 개념
  - 여러 분류기들을 모아 하나의 분류기를 만들어 이를 메타 분류기라고 부름
- 시간이 굉장히 오래 걸리지만 비교적 좋은 성능을 냄


- 하나의 데이터를 넣음
  - 이를 여러 모델에 학습시키고
  - 테스트 데이터를 각 모델에 입력
  - 투표 또는 여러 가중치 기법을 적용하여 최종 선택
- 바닐라 앙상블
  - 가장 기본적인 앙상블 기법, 아무것도 처리하지 않은 앙상블 모델을 의미
  - 일반적으로 가중치 평균이나 투표 방식으로 만들어지는 앙상블 모델
- 부스팅
  - 하나의 모델에서 여러 데이터를 샘플링한 다음 그 샘플링된 데이터로 각각의 모델을 만드는 기법
- 배깅
  - boosting aggregation(부스팅 집합)의 줄임말로 부스팅을 좀 더 발전시킨 기법


# 투표 분류기의 개념

- 투표 분류기(voting classifier)
  - 여러 개의 모델을 만들어 모두 같은 데이터를 넣고 결과를 취합하여 가장 많이 선택된 결과를 취함
  - 앙상블 모델의 가장 기본적인 형태
  - 다수결 분류기(majority voting classifier)라고도 부름
  - 또는 각 분류기마다 가중치를 주고 해당 가중치를 각 모델에 곱하여 가중치의 합을 구하는 방식


# 투표 분류기의 클래스

- 투표 분류기의 클래스
  - 사이킷런에서 제공하는 VotingClassifier 클래스 사용


#### 학습


In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

pd.set_option("future.no_silent_downcasting", True)


def transform_status(x):
    if "Mrs" in x or "Ms" in x:
        return "Mrs"
    elif "Mr" in x:
        return "Mr"
    elif "Miss" in x:
        return "Miss"
    elif "Master" in x:
        return "Master"
    elif "Dr" in x:
        return "Dr"
    elif "Rev" in x:
        return "Rev"
    elif "Col" in x:
        return "Col"
    else:
        return "0"


train_df = pd.read_csv("./titanic/train.csv")
test_df = pd.read_csv("./titanic/test.csv")

train_id = train_df["PassengerId"].values
test_id = test_df["PassengerId"].values

all_df = pd.concat([train_df, test_df]).set_index("PassengerId")
all_df["Sex"] = all_df["Sex"].replace({"male": 0, "female": 1}).astype(int)
all_df["Age"] = all_df["Age"].fillna(all_df.groupby("Pclass")["Age"].transform("mean"))
all_df["cabin_count"] = all_df["Cabin"].map(
    lambda x: len(x.split()) if type(x) == str else 0
)
all_df["social_status"] = all_df["Name"].map(lambda x: transform_status(x))
all_df = all_df.drop([62, 830])
train_id = np.delete(train_id, [62 - 1, 830 - 1])
all_df.loc[all_df["Fare"].isnull(), "Fare"] = 12.415462
all_df["cabin_type"] = all_df["Cabin"].map(lambda x: x[0] if type(x) == str else "99")

del all_df["Cabin"]
del all_df["Name"]
del all_df["Ticket"]

y = all_df.loc[train_id, "Survived"].values
del all_df["Survived"]

X_df = pd.get_dummies(all_df)
X = X_df.values

minmax_scaler = MinMaxScaler()
minmax_scaler.fit(X)
X = minmax_scaler.transform(X)

X_train = X[: len(train_id)]
X_test = X[len(train_id) :]

np.save("./titanic/titanic_X_train.npy", X_train)
np.save("./titanic/titanic_y_train.npy", y)
np.save("./titanic/titanic_test.npy", X_test)

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

- 전처리되어 .npy 파일 형태인 데이터를 호출


In [39]:
X = np.load("./titanic/titanic_X_train.npy")
y = np.load("./titanic/titanic_y_train.npy")

In [40]:
X[0]

array([1.        , 0.        , 0.27345609, 0.125     , 0.        ,
       0.01415106, 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [41]:
y[:10]

array([0., 1., 1., 1., 0., 0., 0., 0., 1., 1.])

In [29]:
clf1 = LogisticRegression(random_state=1)
clf2 = DecisionTreeClassifier(random_state=1, max_depth=4)
clf3 = GaussianNB()

eclf = VotingClassifier(
    estimators=[("lr", clf1), ("rf", clf2), ("gnb", clf3)], voting="hard"
)

# 투표 분류기의 클래스

- 투표 분류기의 클래스
  - 투표 분류기의 성능과 모델별 성능을 측정
  - clf3 제외하면 전체 모델 성능보다 개별 모델 성능이 높음


In [42]:
from sklearn.model_selection import cross_val_score

cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8222941661905668)

In [43]:
cross_val_score(clf1, X, y, cv=5).mean()

np.float64(0.8290420872214816)

In [44]:
cross_val_score(clf2, X, y, cv=5).mean()

np.float64(0.8223068621849807)

In [45]:
cross_val_score(clf3, X, y, cv=5).mean()

np.float64(0.4600139655938551)

# 투표 분류기의 클래스

- 투표 분류기의 클래스
  - GaussianNB는 연속적인 데이터를 다루기 위한 모델로 데이터셋과 맞지 않아 해당 모델을 빼고 성능을 측정
  - 앙상블 모델에서는 반드시 많은 수의 모델 조합이 가장 최선의 결과를 내는 것은 아님


In [46]:
eclf = VotingClassifier(estimators=[("lr", clf1), ("rf", clf2)], voting="hard")
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8301783787215135)

# 하이퍼 매개변수를 튜닝한 투표 분류기

- 하이퍼 매개변수를 튜닝한 투표 분류기
  - 성능이 좋았던 모델 두 개를 각각 VotingClassifier에 할당


In [47]:
clf1 = LogisticRegression(random_state=1)
clf2 = DecisionTreeClassifier(random_state=1)
eclf = VotingClassifier(estimators=[("lr", clf1), ("dt", clf2)], voting="hard")

In [48]:
c_params = [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]

params = {
    "lr__solver": ["liblinear"],
    "lr__penalty": ["l2"],
    "lr__C": c_params,
    "dt__criterion": ["gini", "entropy"],
    "dt__max_depth": [10, 8, 7, 6, 5, 4, 3, 2],
    "dt__min_samples_leaf": [1, 2, 3, 4, 5, 6, 7, 8, 9],
}

- 가장 좋은 모델의 성능을 확인


In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X, y)
grid.best_score_

np.float64(0.8425569732749316)

- 가장 좋은 성능을 내는 매개변수 확인


In [50]:
grid.best_params_

{'dt__criterion': 'gini',
 'dt__max_depth': 10,
 'dt__min_samples_leaf': 5,
 'lr__C': 5.0,
 'lr__penalty': 'l2',
 'lr__solver': 'liblinear'}